# MRathon 2024 - NeuroLibre preprint project

Hello world!


## Figure 1

In [ ]:
import numpy as np
from scipy.fft import fftn, ifftn, fftshift
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display, HTML
from plotly.offline import init_notebook_mode, iplot, plot

init_notebook_mode(connected=True)
vol_size = (128, 128, 128)
vol_centre = (vol_size[1] // 2, vol_size[2] // 2)
offset = (10, 20)
radius_inner = 10
radius_outer = 40
cylinder_base = np.zeros(vol_size)
cylinder_internal = np.zeros(vol_size)

x = np.arange(-vol_size[1]//2,vol_size[1]//2,dtype=float)
y = x
z = x
X,Y,Z = np.meshgrid(x,y,z)

cylinder_base[X**2+Y**2<=radius_outer**2] = 1

cylinder_internal[(X-offset[0])**2+(Y-offset[1])**2<=radius_inner**2] += 0.1

kx = X
ky = Y
kz = Z
KK = (kx**2 + ky**2 + kz**2)
KK[KK==0] = np.nan
dipole_kernel = 2./3 - (kx**2)/(KK)
dipole_kernel[(kx**2 + ky**2 + kz**2) == 0] = 0

fieldmap = np.real(ifftn(fftshift(dipole_kernel) * (fftn(cylinder_internal))))

idx = 64

fig = make_subplots(rows=1, cols=3, shared_xaxes=False, horizontal_spacing=0.1,
                    subplot_titles=("Geometry",
                                    "Dipole kernel",
                                    "Fieldmap"),
                    specs=[[{"type": "Heatmap"}, {"type": "Heatmap"}, {"type": "Heatmap"}]])
fig.add_trace(go.Heatmap(z=cylinder_base[:,:,idx]+cylinder_internal[:,:,64], colorscale='gray'), 1, 1)
fig.add_trace(go.Heatmap(z=dipole_kernel[:,:,idx], colorscale='gray'), 1, 2)
fig.add_trace(go.Heatmap(z=fieldmap[:,:,idx], colorscale='gray'), 1, 3)

plot(fig, filename = 'geometry.html')



In [ ]:
display(HTML('geometry.html'))

## Figure 2

In [ ]:
fig = go.Figure()

echo_times = [10, 30, 50]
for step in echo_times:
    fig.add_trace(
        go.Heatmap(
            visible=False,
            z=(fieldmap[:,:,idx]*step) % np.pi,
            colorscale='gray'))

fig.data[0].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "TE=" + str(echo_times[i]) + " ms"}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Frequency: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

plot(fig, filename = 'cylinder.html')

In [ ]:
display(HTML('cylinder.html'))